### 練習問題2.65
練習問題2.63と練習問題2.64の結果を使って、  
(バランスの取れた)⼆分木として実装された集合に対する  
union-setとintersection-setを$\Theta(n)$で実装せよ。 

In [1]:
(define (entry tree) (car tree))
(define (left-branch tree) (cadr tree))
(define (right-branch tree) (caddr tree))
(define (make-tree entry left right) (list entry left right))
(define (element-of-set? x set)
  (cond ((null? set) #f)
        ((= x (entry set)) #t)
        ((< x (entry set)) (element-of-set? x (left-branch set)))
        ((> x (entry set)) (element-of-set? x (right-branch set)))
        )
  )

(define (list->tree elements)
  (car (partial-tree elements (length elements)))
  )
(define (partial-tree elts n)
  (if (= n 0) (cons '() elts)
      (let ((left-size (quotient (- n 1) 2)))
        (let ((left-result (partial-tree elts left-size)))
          (let ((left-tree (car left-result))
                (non-left-elts (cdr left-result)) (right-size (- n (+ left-size 1))))
            (let ((this-entry (car non-left-elts))
                  (right-result (partial-tree (cdr non-left-elts) right-size)))
              (let ((right-tree (car right-result))
                    (remaining-elts (cdr right-result)))
                (cons (make-tree this-entry left-tree right-tree) remaining-elts)
                )              
              )
            )
          )
        )
    )
) 

(define (tree->list tree)
    (if (null? tree) '()
        (append
            (tree->list (left-branch tree))
            (cons (entry tree) (tree->list (right-branch tree)))
        )
    )
)

In [2]:
(list->tree '(1 3 5 7 9 11))

(5 (1 () (3 () ())) (9 (7 () ()) (11 () ())))

方針
・2つの二分木をリストに順序付きリストa、bに変換する。
・a、bから順序付きリストによる共通集合、和集合を生成する。
・順序付きリストからlist->tree手続きを使って二分木に変換する。

In [3]:
(define (element-of-set? x set)
  (cond ((null? set) #f)
        ((= x (car set)) #t)
        ((< x (car set)) #f)
        (else (element-of-set? x (cdr set))))
  )

(define (intersection-set tree1 tree2)
  (define (iter set1 set2)
    (if (or (null? set1) (null? set2)) '()
        (let ((x1 (car set1))
              (x2 (car set2)))
          (cond ((= x1 x2) (cons x1 (iter (cdr set1) (cdr set2))))
                ((< x1 x2) (iter (cdr set1) set2))
                ((< x2 x1) (iter set1 (cdr set2)))
            )
          )
        )
    )
  (let ((set1 (tree->list tree1))
        (set2 (tree->list tree2)))
    (let ((set (iter set1 set2)))
      (list->tree set)
      )
    )
  )

(define (union-set tree1 tree2)
  (define (iter set1 set2)
    (cond ((and (null? set1) (null? set2)) '())
          ((and (not (null? set1)) (null? set2)) set1)
          ((and (null? set1) (not (null? set2))) set2)
          (else
           (let ((x1 (car set1))
                 (x2 (car set2)))
             (cond ((= x1 x2) (cons x1 (iter (cdr set1) (cdr set2))))
                   ((< x1 x2) (cons x1 (iter (cdr set1) set2)))
                   ((< x2 x1) (cons x2 (iter set1 (cdr set2))))
               )
            )
           )
          )
       )
  (let ((set1 (tree->list tree1))
        (set2 (tree->list tree2)))
    (let ((set (iter set1 set2)))
      (list->tree set)
      )
    )
  )

In [4]:
(intersection-set
         '(5 (1 () (3 () ())) (9 (7 () ()) (11 () ())))
         '(7 (4 (1 () ()) (5 () ())) (9 () (12 () ())))
 )

(5 (1 () ()) (7 () (9 () ())))

In [5]:
(union-set
         '(5 (1 () (3 () ())) (9 (7 () ()) (11 () ())))
         '(7 (4 (1 () ()) (5 () ())) (9 () (12 () ())))
 )

(5 (3 (1 () ()) (4 () ())) (9 (7 () ()) (11 () (12 () ()))))